In [26]:
import pandas as pd

DATASET_PATH = './../dataset/Encoded.csv'
LABEL_COLUMN = 'Attack Type'

In [12]:
df = pd.read_csv(DATASET_PATH)

df.head()

,Unnamed: 0,Seq,Dur,RunTime,Mean,Sum,Min,Max,sTos,dTos,...,54,af11,af12,af41,cs0,cs4,cs6,cs7,ef,nan
0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,3,4.998020,4.998020,4.998020,4.998020,4.998020,4.998020,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,4,4.998037,4.998037,4.998037,4.998037,4.998037,4.998037,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,5,4.999453,4.999453,4.999453,4.999453,4.999453,4.999453,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.info()

In [ ]:
df.drop(columns=['Unnamed: 0', 'Seq', 'Attack Tool', 'Label', 'nan'], inplace=True)
df.info()

In [32]:
def calc_percentage_classes(df):
    return df[LABEL_COLUMN].value_counts(normalize=True)*100

calc_percentage_classes(df)

Attack Type
Benign            39.280450
UDPFlood          37.620221
HTTPFlood         11.583020
SlowrateDoS        6.015090
TCPConnectScan     1.649453
SYNScan            1.648712
UDPScan            1.308408
SYNFlood           0.799637
ICMPFlood          0.095009
Name: proportion, dtype: float64

In [16]:
for col in df.columns:
    if df[col].isnull().sum() > (len(df) * 0.7):
        print("dropping", col, df[col].isnull().sum())
        df.drop(columns = [col], inplace=True)

dropping dTos 943067
dropping dTtl 943067
dropping dHops 943067
dropping SrcGap 937219
dropping DstGap 937219
dropping SrcWin 973470
dropping DstWin 1038812
dropping sVid 1101319
dropping dVid 1213881
dropping SrcTCPBase 937219
dropping DstTCPBase 985843


In [ ]:
df = df.dropna()
df.info()

In [ ]:
d = {}
for col in df.columns:
    v = df[col].unique()
    print(col, v)
    if len(v) in [1,2]: #and map(lambda val: val in (0, 1), v):
        d[col] = bool
df = df.astype(d)
#df.info()

In [ ]:
s = df.select_dtypes(include=['float', 'int'])
s

In [ ]:
s = (s - s.mean())/s.std()
for col in s: #df.select_dtypes(include='category').columns:
    df[col] = s[col]
df

In [31]:
d = {}
for col in df.columns:
    v = df[col].unique()
    if len(v) in [1,2]: #and map(lambda val: val in (0, 1), v):
        d[col] = int
df = df.astype(d)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1215676 entries, 0 to 1215889
Data columns (total 80 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Dur          1215676 non-null  float64
 1   RunTime      1215676 non-null  float64
 2   Mean         1215676 non-null  float64
 3   Sum          1215676 non-null  float64
 4   Min          1215676 non-null  float64
 5   Max          1215676 non-null  float64
 6   sTos         1215676 non-null  float64
 7   sTtl         1215676 non-null  float64
 8   sHops        1215676 non-null  float64
 9   TotPkts      1215676 non-null  float64
 10  SrcPkts      1215676 non-null  float64
 11  DstPkts      1215676 non-null  float64
 12  TotBytes     1215676 non-null  float64
 13  SrcBytes     1215676 non-null  float64
 14  DstBytes     1215676 non-null  float64
 15  Offset       1215676 non-null  float64
 16  sMeanPktSz   1215676 non-null  float64
 17  dMeanPktSz   1215676 non-null  float64
 18  Load   

In [24]:
def reduce_class(df, col_name, class_val, p):
    indices = df.index[df[col_name] == class_val]
    return df.drop(indices[:int(len(indices) * p)])

In [50]:
x = reduce_class(df, LABEL_COLUMN, 'Benign', 0.4)
x = reduce_class(x, LABEL_COLUMN, 'UDPFlood', 0.5)
x = reduce_class(x, LABEL_COLUMN, 'HTTPFlood', 0.33)
x = reduce_class(x, LABEL_COLUMN, 'SlowrateDoS', 0.15)
x = reduce_class(x, LABEL_COLUMN, 'ICMPFlood', 1)

calc_percentage_classes(x)

Attack Type
Benign            38.854256
UDPFlood          31.010012
HTTPFlood         12.794156
SlowrateDoS        8.428995
TCPConnectScan     2.719258
SYNScan            2.718038
UDPScan            2.157018
SYNFlood           1.318268
Name: proportion, dtype: float64